<a href="https://colab.research.google.com/github/polyankaglade/RuCor2SpanBert/blob/master/conll2spanbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/polyankaglade/coref.git

Cloning into 'coref'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 734 (delta 2), reused 0 (delta 0), pack-reused 728
Receiving objects: 100% (734/734), 4.17 MiB | 17.10 MiB/s, done.
Resolving deltas: 100% (441/441), done.


In [1]:
%cd /content/coref

/content/coref


In [ ]:
!pip install -r requirements.txt

In [2]:
! export data_dir=./data

Some `g++` versions may not play nicely with this repo. If you get this:
`tensorflow.python.framework.errors_impl.NotFoundError: ./coref_kernels.so: undefined symbol: _ZN10tensorflow12OpDefBuilder4AttrESs`, try removing the flag `-D_GLIBCXX_USE_CXX11_ABI=0` from `setup_all.sh`. Thanks to Naman Jain for the [solution](https://github.com/mandarjoshi90/coref/issues/29).


In [3]:
! ./setup_all.sh

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [4]:
!mkdir Data

mkdir: cannot create directory ‘Data’: File exists


In [16]:
## положите <train, test, dev>.russian.v4_gold_conll в Data

In [4]:
import re
import os
import sys
import json
import tempfile
import subprocess
import collections


import util as util
import conll as connl
from bert import tokenization

#########

from minimize import minimize_language


vocab_file = './cased_config_vocab/vocab.txt'
input_dir = './Data'
output_dir = './Output'
do_lower_case = False
print(do_lower_case)
labels = collections.defaultdict(set)
stats = collections.defaultdict(int)
if not os.path.isdir(output_dir):
  os.mkdir(output_dir)
for seg_len in [128, 256]:
  minimize_language("english", labels, stats, vocab_file, seg_len, input_dir, output_dir, do_lower_case)
  # minimize_language("chinese", labels, stats, vocab_file, seg_len)
  # minimize_language("es", labels, stats, vocab_file, seg_len)
  # minimize_language("arabic", labels, stats, vocab_file, seg_len)
for k, v in labels.items():
  print("{} = [{}]".format(k, ", ".join("\"{}\"".format(label) for label in v)))
for k, v in stats.items():
  print("{} = {}".format(k, v))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

False
Minimizing ./Data/dev.english.v4_gold_conll
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen very often.)
Merging clusters (shouldn't happen ver

In [18]:
def convert_mention(output, mention, comb_text):
    start = output['subtoken_map'][mention[0]]
    end = output['subtoken_map'][mention[1]] + 1
    nmention = (start, end)
    mtext = ''.join(' '.join(comb_text[mention[0]:mention[1]+1]).split(" ##"))
    return (nmention, mtext)

def get_clusters(output):
    comb_text = [word for sentence in output['sentences'] for word in sentence]
    seen = set()
    #print('Clusters:')
    clusters = []
    for cluster in output['clusters']:
        mapped = []
        for mention in cluster:
            seen.add(tuple(mention))
            mapped.append(convert_mention(output, mention, comb_text))
        clusters.append(mapped)

    return clusters

In [7]:
pip install jsonlines

In [8]:
import jsonlines

In [12]:
with jsonlines.open('./Output/train.english.256.jsonlines') as reader:
    x = [o for o in reader]

In [13]:
len(x)

181

In [16]:
x[0]['sentences'][0]

['[CLS]',
 'В',
 '##о',
 'в',
 '##р',
 '##е',
 '##м',
 '##я',
 'с',
 '##в',
 '##о',
 '##и',
 '##х',
 'п',
 '##р',
 '##о',
 '##г',
 '##у',
 '##л',
 '##о',
 '##к',
 'в',
 'о',
 '##к',
 '##р',
 '##е',
 '##с',
 '##т',
 '##н',
 '##о',
 '##с',
 '##т',
 '##я',
 '##х',
 'С',
 '##и',
 '##м',
 '##е',
 '##и',
 '##з',
 '##а',
 'я',
 'о',
 '##б',
 '##р',
 '##а',
 '##т',
 '##и',
 '##л',
 'в',
 '##н',
 '##и',
 '##м',
 '##а',
 '##н',
 '##и',
 '##е',
 'н',
 '##а',
 'о',
 '##д',
 '##и',
 '##н',
 '##о',
 '##к',
 '##у',
 '##ю',
 'д',
 '##а',
 '##ч',
 '##у',
 ',',
 'с',
 '##т',
 '##о',
 '##я',
 '##в',
 '##ш',
 '##у',
 '##ю',
 'н',
 '##а',
 'к',
 '##р',
 '##у',
 '##т',
 '##о',
 '##м',
 'с',
 '##к',
 '##л',
 '##о',
 '##н',
 '##е',
 'г',
 '##о',
 '##р',
 '##ы',
 '.',
 'К',
 'э',
 '##т',
 '##о',
 '##й',
 'д',
 '##а',
 '##ч',
 '##е',
 'н',
 '##е',
 'б',
 '##ы',
 '##л',
 '##о',
 'п',
 '##р',
 '##ов',
 '##е',
 '##д',
 '##е',
 '##н',
 '##о',
 'д',
 '##а',
 '##ж',
 '##е',
 'д',
 '##о',
 '##р',
 '##о',
 '##г',
 '##и

In [19]:
predictions = [get_clusters(output) for output in x]

In [20]:
clusters = [[[m[1] for m in cluster] for cluster in clusters] for clusters in predictions]
clusters[1]

[['я',
  'я',
  'Елены Сергеевны',
  'Сама',
  'вы',
  'ваших',
  'вы',
  'Елены Сергеевны',
  'Елена Сергеевна',
  'мой',
  'вы',
  'Сергеевой',
  'нею',
  'Елена Сергеевна',
  'Старая учительница',
  'учительницей',
  'Сергеева',
  'Елены Сергеевны',
  'вам',
  'ее',
  'вы',
  'она',
  'вас',
  'Сергеева',
  'Мне',
  'нею',
  'Мои',
  'нее',
  'Я',
  'она',
  'те',
  'те',
  'те',
  'она',
  'те',
  'вы',
  'мне',
  'Елена Сергеевна',
  'Она',
  'Я',
  'соседкой',
  'Я',
  'вы - то',
  'Она',
  'Елена Сергеевна'],
 ['него',
  'Он',
  'Я',
  'он',
  'он',
  'ты',
  'Дементьев',
  'моей',
  'человек',
  'невысокий молодой человек',
  'Ты',
  'я',
  'я',
  'ты',
  'мне',
  'Дементьев',
  'ты',
  'актер',
  'Дементьев',
  'я',
  'мужчина',
  'он',
  'он',
  'Я',
  'Дементьева',
  'его'],
 ['комната', 'комнату новую'],
 ['их',
  'Они',
  'Соседи',
  'Соседи',
  'ними',
  'себя',
  'ваших троглодитах',
  'Мои соседи',
  'они'],
 ['ней',
  'Ты',
  'шокированная всем его видом Мордатенкова',